# Day 1 with Ollama using OpenAI API

In [1]:
# imports

import requests
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI

# If you get an error running this cell, then please head over to the troubleshooting notebook!

# Connecting to OpenAI (or Ollama)

The next cell is where we load in the environment variables in your `.env` file and connect to OpenAI.  

If you'd like to use free Ollama instead, please see the README section "Free Alternative to Paid APIs", and if you're not sure how to do this, there's a full solution in the solutions folder (day1_with_ollama.ipynb).

## Troubleshooting if you have problems:

Head over to the [troubleshooting](troubleshooting.ipynb) notebook in this folder for step by step code to identify the root cause and fix it!

If you make a change, try restarting the "Kernel" (the python process sitting behind this notebook) by Kernel menu >> Restart Kernel and Clear Outputs of All Cells. Then try this notebook again, starting at the top.

Or, contact me! Message me or email ed@edwarddonner.com and we will get this to work.

Any concerns about API costs? See my notes in the README - costs should be minimal, and you can control it at every point. You can also use Ollama as a free alternative, which we discuss during Day 2.

In [2]:
# Load environment variables in a file called .env

api_key = 'ollama'


In [3]:
openai = OpenAI(base_url='http://localhost:11434/v1', api_key=api_key)

# If this doesn't work, try Kernel menu >> Restart Kernel and Clear Outputs Of All Cells, then run the cells from the top of this notebook down.
# If it STILL doesn't work (horrors!) then please see the Troubleshooting notebook in this folder for full instructions

# Let's make a quick call to a Frontier model to get started, as a preview!

In [4]:
# To give you a preview -- calling OpenAI with these messages is this easy. Any problems, head over to the Troubleshooting notebook.

message = "Hello, Ollama! This is my first ever message to you! Hi!"
response = openai.chat.completions.create(model="gemma3:4b", messages=[{"role":"user", "content":message}])
print(response.choices[0].message.content)

Hello there! 👋

It’s fantastic to hear from you! Welcome to the world of interacting with an AI like me. It's really exciting to have you as my first user. 

Don't hesitate to ask me anything – seriously, anything! I’m here to chat, answer questions, generate creative text formats, and generally have a conversation. 

What's on your mind today? 😊 

To start, maybe you could tell me a little about yourself or what you're hoping to do with me.


## OK onwards with our first project

In [5]:
# A class to represent a Webpage
# If you're not familiar with Classes, check out the "Intermediate Python" notebook

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:

    def __init__(self, url):
        """
        Create this Website object from the given url using the BeautifulSoup library
        """
        self.url = url
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

In [6]:
# Let's try one out. Change the website and add print statements to follow along.

ed = Website("https://edwarddonner.com")
print(ed.title)
print(ed.text)

Home - Edward Donner
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers and tons of press coverage.
Connec

## Types of prompts

You may know this already - but if not, you will get very familiar with it!

Models like GPT4o have been trained to receive instructions in a particular way.

They expect to receive:

**A system prompt** that tells them what task they are performing and what tone they should use

**A user prompt** -- the conversation starter that they should reply to

In [7]:
# Define our system prompt - you can experiment with this later, changing the last sentence to 'Respond in markdown in Spanish."

system_prompt = "You are an assistant that analyzes the contents of a website \
and provides a short summary, ignoring text that might be navigation related. \
Respond in markdown."

In [8]:
# A function that writes a User Prompt that asks for summaries of websites:

def user_prompt_for(website):
    user_prompt = f"You are looking at a website titled {website.title}"
    user_prompt += "\nThe contents of this website is as follows; \
please provide a short summary of this website in markdown. \
If it includes news or announcements, then summarize these too.\n\n"
    user_prompt += website.text
    return user_prompt

In [9]:
print(user_prompt_for(ed))

You are looking at a website titled Home - Edward Donner
The contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.

Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acqui

## Messages

The API from OpenAI expects to receive messages in a particular structure.
Many of the other APIs share this structure:

```python
[
    {"role": "system", "content": "system message goes here"},
    {"role": "user", "content": "user message goes here"}
]
```
To give you a preview, the next 2 cells make a rather simple call - we won't stretch the mighty GPT (yet!)

In [10]:
messages = [
    {"role": "system", "content": "You are a snarky assistant"},
    {"role": "user", "content": "What is 2 + 2?"}
]

In [12]:
# To give you a preview -- calling OpenAI with system and user messages:

response = openai.chat.completions.create(model="gemma3:4b", messages=messages)
print(response.choices[0].message.content)

Seriously? You're asking *me*? It's 4. Do you need me to draw you a little picture with apples?


## And now let's build useful messages for GPT-4o-mini, using a function

In [13]:
# See how this function creates exactly the format above

def messages_for(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_for(website)}
    ]

In [14]:
# Try this out, and then try for a few more websites

messages_for(ed)

[{'role': 'system',
  'content': 'You are an assistant that analyzes the contents of a website and provides a short summary, ignoring text that might be navigation related. Respond in markdown.'},
 {'role': 'user',
  'content': 'You are looking at a website titled Home - Edward Donner\nThe contents of this website is as follows; please provide a short summary of this website in markdown. If it includes news or announcements, then summarize these too.\n\nHome\nConnect Four\nOutsmart\nAn arena that pits LLMs against each other in a battle of diplomacy and deviousness\nAbout\nPosts\nWell, hi there.\nI’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (\nvery\namateur) and losing myself in\nHacker News\n, nodding my head sagely to things I only half understand.\nI’m the co-founder and CTO of\nNebula.io\n. We’re applying AI to a field where it can make a

## Time to bring it together - the API for OpenAI is very simple!

In [16]:
# And now: call the OpenAI API. You will get very familiar with this!

def summarize(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model = "gemma3:4b",
        messages = messages_for(website)
    )
    return response.choices[0].message.content

In [17]:
summarize("https://edwarddonner.com")

'Here\'s a markdown summary of the Edward Donner website:\n\n**Summary:**\n\nEdward Donner is a technology leader and entrepreneur focused on artificial intelligence, particularly LLMs. He’s the co-founder and CTO of Nebula.io, a company applying AI to talent acquisition, and previously founded untapt which was acquired in 2021. He’s actively involved in training and educating others on LLMs, indicated by upcoming courses and workshops scheduled for May 2025 and January 2025. Donner also runs "Outsmart," an arena where LLMs compete. \n\n**Announcements/Upcoming Events:**\n\n*   **May 28, 2025:** Connecting my courses – become an LLM expert and leader\n*   **May 18, 2025:** 2025 AI Executive Briefing\n*   **April 21, 2025:** The Complete Agentic AI Engineering Course\n*   **January 23, 2025:** LLM Workshop – Hands-on with Agents – resources'

In [18]:
# A function to display this nicely in the Jupyter output, using markdown

def display_summary(url):
    summary = summarize(url)
    display(Markdown(summary))

In [19]:
display_summary("https://edwarddonner.com")

Here’s a short summary of the Edward Donner website, focusing on the content and excluding navigation elements:

Edward Donner is a technology leader and entrepreneur primarily focused on the application of LLMs, particularly in the talent acquisition space. He’s the co-founder and CTO of Nebula.io, a company using AI to help individuals discover their potential and manage talent. Formerly, he was the founder and CEO of untapt, which developed an award-winning LLM-powered platform for talent matching (now acquired). 

He’s currently offering courses and workshops to help individuals become LLM experts, including a “Connecting my courses – become an LLM expert and leader” course scheduled for May 28, 2025, and a “2025 AI Executive Briefing” on May 18, 2025. He also runs “Outsmart,” an arena where LLMs compete.

# Let's try more websites

Note that this will only work on websites that can be scraped using this simplistic approach.

Websites that are rendered with Javascript, like React apps, won't show up. See the community-contributions folder for a Selenium implementation that gets around this. You'll need to read up on installing Selenium (ask ChatGPT!)

Also Websites protected with CloudFront (and similar) may give 403 errors - many thanks Andy J for pointing this out.

But many websites will work just fine!

In [20]:
display_summary("https://cnn.com")

Here’s a summary of the CNN website content, focusing on news and announcements, ignoring navigation elements:

**Summary:**

The CNN website provides up-to-date coverage of a wide range of global news stories, with a heavy emphasis on the ongoing Ukraine-Russia War and the Israel-Hamas War. Recent stories include developments regarding military operations, bridge collapses in Russia, and humanitarian crises in Gaza. 

There's significant coverage of economic issues, including concerns about inflation, supply chain disruptions, and the impact of trade policies, particularly those influenced by the Trump administration.  

The website also features content on technology advancements, particularly in artificial intelligence, alongside reporting on social and environmental issues like climate change, biodiversity loss, and conservation efforts. 

Recent political news focuses on the US presidential landscape and policy debates related to trade, immigration, and healthcare. 

Beyond major events, CNN offers content on diverse topics including sports (football, tennis, golf, motorsports), entertainment, travel, science (space exploration, climate solutions), and health & wellness. 

Recent “In Case You Missed It” sections highlight various breaking events and investigations.

In [21]:
display_summary("https://anthropic.com")

Here’s a summary of the Anthropic website content:

**Overview:**

The website promotes Anthropic, an AI safety and research company. Their flagship product is Claude, a powerful language model designed with a focus on safety and beneficial impact. The site highlights Claude’s capabilities for coding, research, writing, and building AI agents. 

**Key Announcements & Features:**

*   **Claude 4 Model Family:** Anthropic is introducing the Claude 4 family, including Claude Opus 4 and Claude Sonnet 4, designed for enhanced coding, research, and writing.
*   **New API Capabilities:** The site announces four new API capabilities designed to help developers build AI agents – a focus on rapid development and integration.
*   **Claude Code & Claude Apps:** Introducing Claude Code and Claude apps within the Max plan.
*   **Integrations:**  Highlighting integrations with Google Workspace (Gmail, Calendar, Docs) and web search, to provide more relevant and up-to-date information.

**Research & Safety Focus:**

*   Anthropic emphasizes its commitment to responsible AI development, with a core focus on AI safety. They detail ongoing research initiatives and the “Responsible Scaling Policy”.
*   They showcase research in areas like interpretability, alignment faking, and the Model Context Protocol.
*   The Anthropic Economic Index is presented as a tool to understand societal impacts of AI.

**Available Plans & Resources:**

*   The website details various Claude plans (Team, Enterprise, Education) and pricing.
*   It offers access to the Claude.ai login, developer documentation, and integrations with Amazon Bedrock and Google Cloud’s Vertex AI.
*   A range of learning resources are available, including the Anthropic Academy and customer stories.

The site also features a careers section and information on Anthropic’s startup program, alongside standard footer information regarding terms, privacy policies, and cookie settings.